In [ ]:
import xarray as xr
import hvplot.xarray
import panel as pn
import pandas as pd
from holoviews.element import tiles
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs
import fsspec
import numpy as np
import glob
import dask
import geopandas as gpd
import re
gv.extension('bokeh',logo=False)
hv.extension('bokeh',logo=False)
pn.extension()

In [ ]:
from dask_gateway import Gateway
from dask.distributed import Client
gateway = Gateway()
options = gateway.cluster_options()
cluster = gateway.new_cluster()
cluster.adapt(minimum=0,maximum=80)
client = Client(cluster)
client

In [ ]:
cper = gpd.read_file('data/cper.geojson')
cper = cper.rename(columns={'Past_Name_':'Pasture'})
df = pd.read_csv('https://hls.gsfc.nasa.gov/wp-content/uploads/2016/10/S2_TilingSystem2-1.txt',delim_whitespace=True)
min_geo_x,min_geo_y,max_geo_x,max_geo_y = cper.to_crs(4326).total_bounds
lon = (min_geo_x+max_geo_x)/2.
lat = (min_geo_y+max_geo_y)/2.
tile_info = df[(df.MinLon<=lon)&(df.MaxLon>=lon)&(df['MinLon.1']<=lat)&(df['MaxLon.1']>=lat)]
tile_id = tile_info.TilID.values[0]
epsg = int(tile_info.EPSG.values[0])
min_x,min_y,max_x,max_y = cper.to_crs(epsg).total_bounds
total_area = cper.area.sum()/(1000.*1000.) #Square km

In [ ]:
fs = fsspec.filesystem('https')
S30_yrs = np.arange(2015,2020,dtype=int)
L30_yrs = np.arange(2013,2020,dtype=int)

tile_id1,tile_id2,tile_id3,tile_id4 = [l for l in re.split('(\d*)',tile_id) if l is not '']

# Get list of all Files
f_names = {'Sentinel':[],
           'Landsat':[]}
S_prefix = 'https://hlssa.blob.core.windows.net/hls/S309/'
L_prefix = 'https://hlssa.blob.core.windows.net/hls/L309/'
for yr in S30_yrs:
    for fi in fs.glob('https://hls.gsfc.nasa.gov/data/v1.4/S30/'+str(yr)+'/'+tile_id1+'/'+tile_id2+'/'+tile_id3+'/'+tile_id4+'/*.hdf'):
        bname = glob.os.path.basename(fi)[0:-4]
        f_names['Sentinel'].append(S_prefix+bname)
for yr in L30_yrs:
    for fi in fs.glob('https://hls.gsfc.nasa.gov/data/v1.4/L30/'+str(yr)+'/'+tile_id1+'/'+tile_id2+'/'+tile_id3+'/'+tile_id4+'/*.hdf'):
        bname = glob.os.path.basename(fi)[0:-4]
        f_names['Landsat'].append(L_prefix+bname)

In [ ]:
#Read Data from the Azure Blob Storage
def read_dat(blob_file):
    try:
        if 'S30' in blob_file:
            red = xr.open_rasterio(blob_file+'_04.tif',chunks={}).sel(y=slice(max_y,min_y),x=slice(min_x,max_x))
            nir = xr.open_rasterio(blob_file+'_09.tif',chunks={}).sel(y=slice(max_y,min_y),x=slice(min_x,max_x))
        if 'L30' in blob_file:
            red = xr.open_rasterio(blob_file+'_04.tif',chunks={}).sel(y=slice(max_y,min_y),x=slice(min_x,max_x))
            nir = xr.open_rasterio(blob_file+'_05.tif',chunks={}).sel(y=slice(max_y,min_y),x=slice(min_x,max_x))
        ndvi = (nir - red) / (nir + red)
        ndvi = ndvi.rename(band='date')
        ndvi = ndvi.assign_coords(date = [pd.to_datetime(blob_file[60:-5],format='%Y%j')])
        ndvi = ndvi.compute()
        return(ndvi)
    except:
        return()

In [ ]:
#Download Data and combine into a single xarray object
ndvi = []
for files in f_names['Sentinel']:
    ndvi.append(dask.delayed(read_dat)(files))
for files in f_names['Landsat']:
    ndvi.append(dask.delayed(read_dat)(files))
ndvi = dask.compute(*ndvi)
ndvi = xr.combine_by_coords([n.to_dataset(name='ndvi') for n in ndvi if n is not ()])

In [ ]:
cper_outline = gv.Polygons(cper.to_crs(epsg),vdims='Pasture',crs=ccrs.epsg(epsg)).opts(tools=['hover'], line_color='k',fill_alpha=0)
pp = pn.Row(ndvi.hvplot.image(x='x',
                              y='y',
                              crs=32613,
                              cmap='viridis',
                              min_width=1000,
                              min_height=800).opts(alpha=.7,clim=(-.2,.6)) * cper_outline * gv.tile_sources.EsriImagery())
pp.servable()